# Implementação no Python

## Fazendo uma requisição

#### User agents

Ao tentar fazer uma série de requisições, recebi o código 403, ou seja, o servidor está bloqueando meu acesso. Uma razão para isso pode ser o fato de não ter sido definido um user-agent logo no início. Ou seja, um cabeçalho enviado junto com a requisição que faz com que ela soe "mais humana". Site para coletar user-agents: https://user-agents.net/

In [66]:
urls = ['https://g1.globo.com/rss/g1/educacao/',
        'https://noticias.uol.com.br/sitemap/v2/today.xml',
        'https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml']

for url in urls:
    response = request_url(url)

URL requisitada com sucesso.
URL requisitada com sucesso.
URL requisitada com sucesso.


## Importação de pacotes

In [169]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm # mostra a barra de carregamento
from datetime import datetime

## Funções

### `request_url`

In [202]:
def request_url(url, print_status=True):
    bad_response_cnt = 0

    headers = [
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15 (Applebot/0.1; +http://www.apple.com/go/applebot) [ip:17.241.227.148]"},
        {"User-Agent": "Mozilla/5.0 (Android 12; Mobile; rv:138.0) Gecko/138.0 Firefox/138.0 [ip:178.197.214.27]"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15 (Applebot/0.1; +http://www.apple.com/go/applebot) [ip:17.241.227.148]"},
        {"User-Agent": "TuneIn Radio Pro/31.9.0; iPhone17,1; iOS/18.5"}
              ]

    user_message = {
        200: 'URL requisitada com sucesso.',
        201: 'Recurso criado com sucesso.',
        202: 'Requisição aceita para processamento.',
        204: 'Requisição bem-sucedida, sem conteúdo retornado.',
        301: 'Recurso movido permanentemente.',
        302: 'Recurso movido temporariamente.',
        400: 'Requisição inválida.',
        401: 'Não autorizado. Autenticação necessária.',
        403: 'Proibido. Você não tem permissão para acessar este recurso.',
        404: 'Recurso não encontrado.',
        405: 'Método HTTP não permitido para o recurso.',
        408: 'Tempo de requisição esgotado.',
        409: 'Conflito. O recurso já existe ou há um problema de estado.',
        500: 'Erro interno do servidor.',
        501: 'Funcionalidade não implementada no servidor.',
        502: 'Bad Gateway. Erro de comunicação entre servidores.',
        503: 'Serviço indisponível. Tente novamente mais tarde.',
        504: 'Gateway Timeout. Tempo limite ao esperar resposta do servidor.'
    }

    # a cada vez que uma requisição falha, tenta-se novamente com um novo user-agent para evitar bloqueios
    while bad_response_cnt < len(headers) - 1:
        response = requests.get(url, headers=headers[bad_response_cnt])
        requisition_status = response.status_code // 100

        match requisition_status:
            case 2 | 3:
                if print_status:
                    print(user_message[response.status_code])
                return response

            case 4 | 5:
                bad_response_cnt += 1
                sleep(0.5)

    print(user_message[response.status_code])
    return response

### `save_data`

In [203]:
def save_data(data:dict):
    df = pd.DataFrame(data)
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

### `save_df`

In [204]:
def save_df(df):
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

### `scrape_uol()`

In [205]:
def scrape_uol(number_of_news=10):
    """
    Função cujo objetivo é coletar urls e datas a partir do site-maps, e salvar as n notícias mais recentes
    num data frame.
    Dessas n mais recentes, uma nova requisição será feita para coletar o título da matéria.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
    """

    target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'UOL',
            "Link da matéria": url.loc.text,
            "Título da matéria": "",
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    news_df = collect_recent_news(news_dict, number_of_news)

    print(f'Processando urls - UOL')
    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)

        if response:
            soup = bs(response.content, "html.parser")
            title = soup.select('meta[property="og:title"]')[0][
                'content']  # o resultado retornado é uma lista com 1 elemento
            news_df.loc[i, "Título da matéria"] = title

    print('Urls processadas com sucesso.')
    return news_df

### `collect_recent_news`

In [226]:
def collect_recent_news(data: dict, n=10):
    df = pd.DataFrame(data)
    df['Data (em ISO)'] = pd.to_datetime(df['Data (em ISO)'], format='ISO8601')
    df.sort_values('Data (em ISO)', ascending=False)

    return df[0:n]

### `scrape_g1`

In [207]:
def scrape_g1(number_of_news=10):
    """
    Função cujo objetivo é coletar urls e datas a partir do site-maps, e salvar as n notícias mais recentes
    num data frame.
    Dessas n mais recentes, uma nova requisição será feita para coletar o título da matéria.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
    """

    target_url = 'https://g1.globo.com/rss/g1/educacao/'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'G1',
            "Link da matéria": item.link.text,
            "Título da matéria": item.title.text,
            "Subtítulo": "",
            "Data (em ISO)": item.pubDate.text
        }
        for item in soup.find_all("item")]

    # Formatar as datas no formato ISO 
    print(f'Processando urls - G1')
    for i in range(len(news_dict)):
        date = news_dict[i]['Data (em ISO)']
        date_dt = datetime.strptime(date, '%a, %d %b %Y %H:%M:%S %z')
        date_iso = date_dt.isoformat()
        news_dict[i]['Data (em ISO)'] = date_iso
    
    news_df = collect_recent_news(news_dict, number_of_news)

    # coletar os subtítulos
    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)
        if response.status_code == 200:
            soup = bs(response.content, "html.parser")
            subtitle = soup.select('meta[property="og:description"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
            news_df.loc[i, "Subtítulo"] = subtitle

    print('Urls processadas com sucesso.')

    return news_df

### `scrape_r7`

In [208]:
def scrape_r7(number_of_news=10):
    target_url = 'https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'R7',
            "Link da matéria": url.loc.text,
            "Título da matéria": url.find('news:title').text.strip(),
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    news_df = collect_recent_news(news_dict, number_of_news)

    print(f'Processando urls - R7')
    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)

        if response:
            soup = bs(response.content, "html.parser")
            subtitle = soup.select('meta[property="og:description"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
            news_df.loc[i, "Subtítulo"] = subtitle

    print('Urls processadas com sucesso.')
    return news_df

## Raspando dados - `noticias.uol.com.br/sitemap/v2/today.xml`

Coletando dados das tags \<url>

In [42]:
target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
response = request_url(target_url)  # coletando o xml do uol

soup = bs(response.content, "xml")

urls_data = [
    {
        "url": url.loc.text,
        "title": "",
        "subtitle": "",
        "ISO_datetime": url.lastmod.text
    }
    for url in soup.find_all("url")]

urls_data[0:2]

https://noticias.uol.com.br/sitemap/v2/today.xml status - ok


[{'url': 'https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:58:25.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/rios-em-sao-paulo.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:57:32.000Z'}]

##### Teste - Fazendo uma chave para cada url individualmente e coletar o título

In [43]:
response = verify_status_code(urls_data[0]['url'])
if response:
        soup = bs(response.content, "html.parser")
        # no head estão inclusas as meta tags, o título encontra-se nessa
        # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
        title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento

https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm status - ok


In [67]:
max_error_cnt = 3

print(f'Processando urls...')
for i in tqdm(range(len(urls_data))):
        error_cnt = 0

        while error_cnt < max_error_cnt:
            try:
                response = request_url(urls_data[i]['url'], print_status=False)
                
                if response:
                        soup = bs(response.content, "html.parser")
                        # no head estão inclusas as meta tags, o título encontra-se nessa
                        # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
                        title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
                        urls_data[i]['title'] = title
                        break
            except Exception as e:
                error_cnt += 1
    

urls_data

Processando urls...


100%|████████████████████████████████████████████████████████████████████████████████| 598/598 [04:08<00:00,  2.41it/s]


[{'url': 'https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm',
  'title': 'Moraes nega pedido de defesa de Câmara para anular delação de Mauro Cid',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:58:25.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/rios-em-sao-paulo.htm',
  'title': 'Será que na sua tem? Há mais de 200 rios embaixo das ruas de SP; veja mapa',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:57:32.000Z'},
 {'url': 'https://noticias.uol.com.br/ultimas-noticias/reuters/2025/06/27/ruanda-e-congo-assinam-acordo-de-paz-nos-eua-para-por-fim-a-conflitos-e-atrair-investimentos.htm',
  'title': 'Ruanda e Congo assinam acordo de paz nos EUA para pôr fim a conflitos e atrair investimentos',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:54:46.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/brf-deve-indenizar-em-r-150-mil-funcionaria-que-perdeu-g

### Resultado final

In [153]:
def scrape_uol(number_of_news=10):
    """
    Função cujo objetivo é coletar urls e datas a partir do site-maps, e salvar as n notícias mais recentes
    num data frame.
    Dessas n mais recentes, uma nova requisição será feita para coletar o título da matéria.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
    """

    target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'UOL',
            "Link da matéria": url.loc.text,
            "Título da matéria": "",
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    news_df = collect_recent_news(news_dict, number_of_news)

    print(f'Processando urls - UOL')
    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)

        if response:
            soup = bs(response.content, "html.parser")
            title = soup.select('meta[property="og:title"]')[0][
                'content']  # o resultado retornado é uma lista com 1 elemento
            news_df.loc[i, "Título da matéria"] = title

    print('Urls processadas com sucesso.')
    return news_df

## Raspando dados -`https://g1.globo.com/rss/g1/educacao/`

In [166]:
target_url = 'https://g1.globo.com/rss/g1/educacao/'
response = request_url(target_url)  # coletando o xml do uol

soup = bs(response.content, "xml")

item_element = soup.find('item')
item_element.title.text

URL requisitada com sucesso.


'Bobbie Goods: técnicas de colorir usadas em livros virais já foram utilizadas por Van Gogh, Botticelli, Caravaggio e outros artistas'

In [196]:
def scrape_g1(number_of_news=10):
    """
    Função cujo objetivo é coletar urls e datas a partir do site-maps, e salvar as n notícias mais recentes
    num data frame.
    Dessas n mais recentes, uma nova requisição será feita para coletar o título da matéria.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
    """

    target_url = 'https://g1.globo.com/rss/g1/educacao/'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'G1',
            "Link da matéria": item.link.text,
            "Título da matéria": item.title.text,
            "Subtítulo": "",
            "Data (em ISO)": item.pubDate.text
        }
        for item in soup.find_all("item")]

    # Formatar as datas no formato ISO 
    print(f'Processando urls - G1')
    for i in range(len(news_dict)):
        date = news_dict[i]['Data (em ISO)']
        date_dt = datetime.strptime(date, '%a, %d %b %Y %H:%M:%S %z')
        date_iso = date_dt.isoformat()
        news_dict[i]['Data (em ISO)'] = date_iso
    
    news_df = collect_recent_news(news_dict, number_of_news)

    # coletar os subtítulos
    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)
        if response.status_code == 200:
            soup = bs(response.content, "html.parser")
            subtitle = soup.select('meta[property="og:description"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
            news_df.loc[i, "Subtítulo"] = subtitle

    print('Urls processadas com sucesso.')

    return news_df

In [197]:
scrape_g1()

URL requisitada com sucesso.
Processando urls - G1
Urls processadas com sucesso.


,Veículo,Link da matéria,Título da matéria,Subtítulo,Data (em ISO)
0,G1,https://g1.globo.com/educacao/noticia/2025/06/...,Bobbie Goods: técnicas de colorir usadas em li...,Adeptos dos livros de colorir têm extravasado ...,2025-06-28 07:00:42+00:00
1,G1,https://g1.globo.com/educacao/noticia/2025/06/...,Prouni abre consulta para bolsas para o 2º sem...,Edição do programa oferece mais de 211 mil bol...,2025-06-27 23:46:47+00:00
2,G1,https://g1.globo.com/sp/sao-paulo/noticia/2025...,Irmãs da Grande SP são premiadas em Londres co...,"As irmãs Beatriz e Isabella Toassa, da",2025-06-27 20:06:46+00:00
3,G1,https://g1.globo.com/educacao/enem/2025/notici...,"Enem 2025: após ampliação do prazo, pagamento ...",Provas serão aplicadas nos dias 9 e 16 de nove...,2025-06-27 03:00:40+00:00
4,G1,https://g1.globo.com/educacao/noticia/2025/06/...,"MEC publica edital do Enamed 2025, exame que a...",As inscrições acontecem de 7 a 18 de julho e d...,2025-06-26 17:40:02+00:00
5,G1,https://g1.globo.com/ciencia/noticia/2025/06/2...,"Bendegó, o meteorito que virou penteado, crôni...",Meteorito virou crônica de Machado de Assis e ...,2025-06-26 11:13:13+00:00
6,G1,https://g1.globo.com/educacao/noticia/2025/06/...,Como professores dos EUA estão usando o ChatGP...,6 em cada 10 professores dos Estados Unidos qu...,2025-06-26 08:01:02+00:00
7,G1,https://g1.globo.com/educacao/noticia/2025/06/...,Brasileira que teve a maior nota da turma de H...,"Brasileira se destacou nos EUA, venceu o maior...",2025-06-26 06:00:15+00:00
8,G1,https://g1.globo.com/educacao/noticia/2025/06/...,EUA anunciam novas regras para visto de estuda...,Medida foi formalizada pela embaixada no Brasi...,2025-06-25 14:00:55+00:00
9,G1,https://g1.globo.com/jogos/g1-jogos-lanca-comb...,g1 jogos lança 'Combinado'; desafio testa raci...,"São 16 palavras divididas em 4 grupos, que com...",2025-06-25 03:00:39+00:00


## Raspando dados - `R7` 

In [247]:
def scrape_r7(number_of_news=10):
    target_url = 'https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml'
    response = request_url(target_url)  # coletando o xml do uol

    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'R7',
            "Link da matéria": url.loc.text,
            "Título da matéria": url.find('news:title').text.strip(),
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    # Formatar as datas no formato ISO 
    # exemplo: 2025-06-28T23:53:48.288Z<
    print(f'Processando urls - R7')

    date_formats = ["%Y-%m-%dT%H:%M:%S", "%Y-%m-%dT%H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%SZ"]
    
    for i in range(len(news_dict)):
        date = news_dict[i]['Data (em ISO)']

        for date_format in date_formats:
            try:
                date_dt = datetime.strptime(date, date_format)
                break
            
            except ValueError:
                continue
        else:
            print(f'Formato não reconhecido para a data: {date}')
            raise Exception
    
        date_iso = date_dt.isoformat()
        news_dict[i]['Data (em ISO)'] = date_iso
        
    news_df = collect_recent_news(news_dict, number_of_news)

    for i in range(len(news_df)):

        response = request_url(news_df['Link da matéria'][i], print_status=False)

        if response:
            soup = bs(response.content, "html.parser")
            subtitle = soup.select('h2.base-font-primary')
            if subtitle: # r7 nem sempre tem subtítulo
                subtitle = subtitle[0].text  # o resultado retornado é uma lista com 1 elemento
                news_df.loc[i, "Subtítulo"] = subtitle

    print('Urls processadas com sucesso.')
    return news_df

In [248]:
r7_df = scrape_r7()

URL requisitada com sucesso.
Processando urls - R7
Urls processadas com sucesso.


In [249]:
r7_df

,Veículo,Link da matéria,Título da matéria,Subtítulo,Data (em ISO)
0,R7,https://www.r7.com/tudo-do-r7/cuidado-voce-pod...,CUIDADO você pode estar sendo um papagaio sem ...,,2025-06-28 23:53:48.288
1,R7,https://www.r7.com/tudo-do-r7/nao-precisa-de-d...,NÃO PRECISA DE DIPLOMA PRA ENTENDER DE DINHEIR...,,2025-06-28 23:51:46.654
2,R7,https://www.r7.com/tudo-do-r7/brasileiros-invi...,Brasileiros invictos! Veja tudo que rolou na 2...,,2025-06-28 05:39:59.687
3,R7,https://www.r7.com/tudo-do-r7/vini-jr-elogia-c...,Vini Jr. elogia campanha de times brasileiros ...,,2025-06-28 05:39:44.151
4,R7,https://www.r7.com/tudo-do-r7/quem-esta-certo-...,Quem está certo nessa história: Neto ou Memphi...,,2025-06-28 05:39:28.451
5,R7,https://www.r7.com/menusidebardestaques-blogs-...,GUSTAVO MENDES FALA SOBRE BARIÁTRICA,,2025-06-27 20:42:56.000
6,R7,https://www.r7.com/tudo-do-r7/ansiedade-financ...,Ansiedade Financeira É Falta De Experiência,,2025-06-27 23:51:55.371
7,R7,https://www.r7.com/tudo-do-r7/gaucha-viraliza-...,Gaúcha viraliza ao relatar ida ao Rinjani: “Ma...,,2025-06-28 05:38:58.827
8,R7,https://www.r7.com/tudo-do-r7/depressao-recorr...,DEPRESSÃO RECORRENTE | ANSIEDADE BRASIL,,2025-06-27 23:50:46.010
9,R7,https://www.r7.com/tudo-do-r7/saiba-como-corta...,Saiba como cortar as unhas dos pets com segurança,,2025-06-27 23:50:11.651


In [237]:
r7_df['Link da matéria'][2]

'https://www.r7.com/tudo-do-r7/brasileiros-invictos-veja-tudo-que-rolou-na-2-rodada-do-mundial-de-clubes-redacaor7-shorts-28062025/'

## Salvando dados no excel

In [91]:
def save_data(data:dict):
    df = pd.DataFrame(data)
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

## Pipeline

In [160]:
uol_df = scrape_uol()
save_df(uol_df)

URL requisitada com sucesso.
Processando urls - UOL
Urls processadas com sucesso.
Dados salvos com sucesso.


## Análise de dados

In [251]:
df = pd.read_csv('result.csv')
df

,UOL,https://noticias.uol.com.br/ultimas-noticias/rfi/2025/06/29/festival-em-deauville-na-franca-homenageia-a-fotografia-esportiva-e-faz-leilao-de-fotos-iconicas.htm,"Festival em Deauville, na França, homenageia a fotografia esportiva e faz leilão de fotos icônicas",Unnamed: 3,2025-06-29 09:29:27+00:00
0,UOL,https://noticias.uol.com.br/newsletters/novos-...,Brasil pode convocar de volta embaixadora se T...,NaN,2025-06-29 08:40:10+00:00
1,UOL,https://noticias.uol.com.br/colunas/jamil-chad...,Jamil Chade: Brasil pode convocar de volta emb...,NaN,2025-06-29 08:31:15+00:00
2,UOL,https://noticias.uol.com.br/internacional/ulti...,Quem foi criminoso que inspirou teoria da sínd...,NaN,2025-06-29 08:31:15+00:00
3,UOL,https://noticias.uol.com.br/newsletters/novos-...,Ditadura prometeu renda a indígenas para liber...,NaN,2025-06-29 08:31:12+00:00
4,UOL,https://noticias.uol.com.br/politica/ultimas-n...,Bolsonaro vai à Paulista com saúde abalada e g...,NaN,2025-06-29 08:30:31+00:00
5,UOL,https://noticias.uol.com.br/politica/ultimas-n...,Proposta de novo Código Civil libera registro ...,NaN,2025-06-29 08:30:29+00:00
6,UOL,https://noticias.uol.com.br/internacional/ulti...,'Jackie Kennedy do Oriente Médio': quem é a úl...,NaN,2025-06-29 08:30:28+00:00
7,UOL,https://noticias.uol.com.br/cotidiano/ultimas-...,Acidente de balão em SC foi o quinto com mais ...,NaN,2025-06-29 08:30:25+00:00
8,UOL,https://noticias.uol.com.br/cotidiano/ultimas-...,Governo planeja 6 novas linhas de trem em 3 re...,NaN,2025-06-29 08:30:23+00:00
9,G1,https://g1.globo.com/educacao/noticia/2025/06/...,Quer estudar fora? Veja 2 dicas da brasileira ...,"Sarah Borges, de 23 anos, foi a primeira brasi...",2025-06-29 07:01:17+00:00


## Testes

In [116]:
iso_str = '2025-06-28T18:10:48.000Z'
data = pd.to_datetime(iso_str)

Timestamp('2025-06-28 18:10:48+0000', tz='UTC')

In [176]:
date = 'Sat, 28 Jun 2025 07:00:42 -0000'
date_dt = datetime.strptime(date, '%a, %d %b %Y %H:%M:%S %z')
date_iso = data_dt.isoformat()
date_iso

'2025-06-28T07:00:42+00:00'